### PromptTemplate
* PromptTemplate
* ChatPromptTemplate
* ChatMessagePromptTemplate
* FewShotPromptTemplate
* PartialPrompt

In [1]:
%pip install -q langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

True

##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT 모델은 대규모 텍스트 데이터셋을 활용하여 자연어 이해 및 생성 능력을 갖추도록 사전 학습됩니다. 이후 새로운 데이터를 '
 '입력받고 이를 이용해 자연어 문장을 생성하며, 이 때 이전에 학습한 지식을 활용하여 새로운 문맥에 적합한 응답을 생성합니다. 이러한 '
 '반복적인 학습과 응답 과정을 통해 모델은 점차 향상되며 사용자의 질문에 대해 더 나은 답변을 제공할 수 있도록 학습됩니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})

pprint(response)

('ChatGPT 모델은 대규모 텍스트 데이터를 활용하여 지도 학습을 통해 다양한 자연어 이해 및 생성 작업을 수행하며, '
 'Transformer 아키텍처를 기반으로 합니다. 모델은 입력 문장을 토큰으로 분리하고, 각 토큰을 임베딩한 후 다양한 층의 '
 '인코더-디코더로 처리하여 자연어 처리 작업을 수행합니다.\n'
 '\n'
 'ChatGPT 모델의 장점으로는 대용량 데이터를 활용하여 다양한 자연어 작업을 수행할 수 있으며, 문맥을 이해하고 다양한 질문에 대답을 '
 '생성할 수 있는 높은 자연어 생성 능력이 있습니다. 또한, 모델의 성능이 지속적으로 발전하여 업데이트가 이루어지고 있습니다.\n'
 '\n'
 '비슷한 AI 모델로는 GPT-3, BERT, RoBERTa, T5 등이 있습니다.')


In [8]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.']
('GPT-4 모델은 대규모의 텍스트 데이터를 학습하여 자연어 이해 및 생성 능력을 강화합니다. 이 모델은 자기 주도적으로 학습하며 문맥과 '
 '의미를 파악하여 다음 단어를 예측하고 생성합니다.')
('Gemma 모델은 입력된 데이터에서 패턴을 학습하여 새로운 데이터를 예측하는 머신러닝 알고리즘입니다.\n'
 '이 모델은 주어진 특성들 간의 상호작용을 학습하고 이를 바탕으로 예측을 수행합니다.\n'
 '학습 과정에서 Gemma 모델은 손실 함수를 최소화하는 방향으로 가중치를 업데이트하여 최적의 예측 모델을 구축합니다.')


##### 2) ChatPromptTemplate
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 튜플 형식 system, user, assistant 메시지 지원
* 간결성과 가독성이 높고 단순한 구조

In [9]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
messages

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]

In [10]:
# 생성한 메시지를 바로 주입하여 호출하기
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
ChatGPT 모델은 GPT (Generative Pre-trained Transformer)을 기반으로 한 언어 생성 모델입니다. ChatGPT 모델이 학습하는 원리는 다음과 같습니다:

1. Self-Attention Mechanism (자기 주의 메커니즘): Transformer 모델의 주요 구성 요소로, 입력 시퀀스 내의 단어들 간의 상호 작용을 학습하는 데 사용됩니다. 이를 통해 모델은 입력 시퀀스의 각 단어의 중요도를 파악하고 더 나은 문맥 파악이 가능해집니다.

2. Pre-training: ChatGPT 모델은 대규모의 텍스트 데이터를 사용하여 사전 훈련되어 있습니다. 이때, 모델은 대화형 데이터를 활용하여 대화를 자연스럽게 이어나갈 수 있는 능력을 배우게 됩니다.

3. Fine-tuning: 실제 사용 사례에 맞게 모델을 세밀하게 튜닝하여, 특정 작업에 가장 잘 맞도록 만듭니다. 예를 들어, ChatGPT 모델을 챗봇으로 사용할 경우, 대화 데이터를 사용하여 모델을 해당 작업에 맞게 미세 조정할 수 있습니다.

4. Generation: 학습된 ChatGPT 모델은 입력된 대화에 기반하여 새로운 텍스트를 생성할 수 있습니다. 이때, 이전에 학습된 대화의 문맥을 고려하여 지속적인 대화가 가능하도록 설계되어 있습니다.

요약하면, ChatGPT 모델은 대규모 데이터를 기반으로 사전 학습된 뒤, 원하는 작업에 맞게 세밀하게 튜닝함으로써 다양한 자연어 생성 작업에 활용될 수 있는 모델입니다.


In [9]:
# 체인을 생성하여 호출하기
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
ChatGPT는 OpenAI에서 개발한 GPT (Generative Pre-trained Transformer) 모델을 기반으로한 챗봇입니다. 이 모델의 학습 원리는 다음과 같습니다:

1. **프리트레이닝 (Pre-training)**: ChatGPT는 대량의 텍스트 데이터를 사용하여 사전 훈련된 모델입니다. 이 모델은 Transformer 아키텍처를 사용하여 언어의 특징을 학습합니다. 프리트레이닝 단계에서 모델은 텍스트의 다음 단어를 예측하는 과제를 수행하면서 언어의 패턴 및 구조를 이해합니다.

2. **미세조정 (Fine-tuning)**: 프리트레이닝 후, ChatGPT는 구체적인 작업에 대해 미세조정됩니다. 이는 사용자가 제공한 데이터세트를 사용하여 특정 작업(챗봇으로의 대화, 질의응답, 등)에 적합하도록 모델을 조정하는 과정입니다. 이를 통해 모델은 특정 작업에 더 잘 맞도록 학습됩니다.

3. **대화 생성 (Dialogue Generation)**: 학습된 ChatGPT 모델은 대화를 생성할 수 있습니다. 입력 문장을 받으면 다음 가장 적합한 단어 또는 문장을 예측하여 대화를 이어나갈 수 있습니다.

4. **자가 회귀적 프로세스 (Autoregressive Process)**: GPT 모델은 입력 시퀀스의 각 단계에서 다음 단어에 대한 확률 분포를 예측하고, 실제 단어를 선택한 후 이전 단계의 출력과 함께 다음 단계의 입력으로 사용하여 계속해서 문장을 생성합니다.

ChatGPT는 자연어 처리(Natural Language Processing) 분야에서 약어의 맥락을 파악하고 자연스러운 대화를 생성하는 데 사용되는 강력한 모델 중 하나입니다.


#### 3) ChatMessagePromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [10]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

Deep learning is a subset of machine learning that is based on artificial neural networks. It involves training neural networks with large amounts of labeled data to learn complex patterns and relationships directly from the data. Deep learning has gained popularity in recent years due to its ability to automatically extract features and patterns from raw data, bypassing the need for manual feature engineering.

Deep learning models, such as deep neural networks, consist of multiple layers of interconnected nodes (neurons) that are organized in a hierarchical manner. Each layer processes the input data and progressively refines the representation of the data as it passes through the network. This hierarchical architecture allows deep learning models to learn intricate patterns and relationships in the data, making them well-suited for a wide range of tasks such as image recognition, speech recognition, natural language processing, and more.

Training deep learning models typically requ

##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

In [11]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")
#result = llm.invoke("양자 얽힘이 무엇인가요?")
print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
태양계에는 수성, 금성, 지구, 화성, 목성, 토성, 천왕성, 해왕성 등 여덟 개의 행성이 있습니다. 수성은 가장 가까운 행성으로, 작은 크기와 뜨거운 온도를 가지고 있습니다. 금성은 지구에 가장 가깝고, 매우 뜨거운 행성으로 유명합니다. 지구는 우리가 사는 행성으로, 생명을 가지고 있는 유일한 행성입니다. 화성은 지구와 가장 비슷한 조건을 가지고 있는 화성이며, 달 표면에도 유사한 모습의 붉은 토양이 산재하고 있습니다. 목성은 태양계에서 가장 큰 행성으로, 가스 행성인 목성의 대기 중엣 풍부한 수소와 헬륨으로 이루어진다. 토성은 가장 큰 링을 가진 행성으로, 아름다운 환상적인 현상을 보여줍니다. 천왕성과 해왕성은 외계행성으로, 천왕성은 가스 행성, 해왕성은 얼음 행성으로 알려져 있습니다.


In [12]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생이 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
# gpt-3.5-turbo
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
#result = chain.invoke({"input": "양자 얽힘이 무엇인가요?"})
print(result.content)

### 태양계의 행성
1. **수성**: 작고 뜨거운 행성으로, 태양에 가장 가까이 위치합니다.
2. **금성**: 지구와 비슷한 크기를 가진 행성으로, 가장 밝게 빛나는 행성입니다.
3. **지구**: 생명체가 서식하는 유일한 행성으로, 물이 풍부하고 다양한 생물이 살고 있습니다.
4. **화성**: 적색 행성으로, 지구와 비슷한 기후를 가지고 있습니다.
5. **목성**: 태양계에서 가장 크고 가장 많은 위성을 가진 행성입니다.
6. **토성**: 아름다운 반지를 가진 행성으로, 가장 많은 반지를 가지고 있습니다.
7. **천왕성**: 층층이 얼어붙은 행성으로, 자전축이 수직에 가깝게 기울어져 있습니다.
8. **해왕성**: 가장 먼 태양계 행성으로, 푸른 색을 띄는 행성입니다.


#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [18]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = model.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 봄에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요? 봄에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 봄에 주로 발생하는 지구과학 현상은 다음과 같습니다.

1. 봄철 대기 순환: 봄철에는 태양의 각도가 증가하면서 남반구에서 북반구로 대류가 발생하게 됩니다. 이로 인해 대기 순환이 활발해지고, 따뜻한 공기와 차가운 공기의 교차로 인해 강력한 바람이 발생할 수 있습니다.

2. 봄철 교차점: 봄철에는 남반구와 북반구의 기온 차이가 크게 줄어들면서 두 지역의 기압 차이가 작아지게 됩니다. 이로 인해 서로 다른 기압대 간의 교차점이 발생하게 되는데, 이는 강력한 대기 순환을 유발할 수 있습니다.

3. 봄철 강수량 변화: 봄철에는 기온이 상승하면서 대기 중의 수증기 양이 증가하게 됩니다. 이로 인해 강수량이 증가하고, 봄철에는 비가 자주 내리는 현상이 발생할 수 있습니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [19]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정 (GPT-4o-mini 사용)
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
#model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# ✅ 모델에 프롬프트 전달 및 응답 받기
response = model.invoke(prompt.format())

# ✅ 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1450.3원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: 현재 1달러가 1450.3원이라는 환율은 여러 가지 경제적 요인에 의해 영향을 받을 수 있습니다. 이 환율을 분석하기 위해 몇 가지 주요 요소를 고려해볼 수 있습니다.

1. **환율의 의미**: 1달러가 1450.3원이라는 것은 한국 원화가 미국 달러에 비해 상대적으로 약세를 보이고 있다는 것을 의미합니다. 이는 수출에 긍정적인 영향을 미칠 수 있지만, 수입 물가 상승으로 이어질 수 있습니다.

2. **경제적 요인**:
   - **금리 차이**: 한국과 미국의 금리 차이가 환율에 큰 영향을 미칠 수 있습니다. 미국의 금리가 높아지면 자본이 미국으로 유입되어 원화가 약세를 보일 수 있습니다.
   - **경제 성장률**: 한국과 미국의 경제 성장률 차이도 환율에 영향을 미칩니다. 한국 경제가 둔화되면 원화가 약세를 보일 가능성이 높습니다.
   - **무역 수지**: 한국의 무역 수지가 흑자일 경우 원화 강세로 이어질 수 있지만, 적자일 경우 원화 약세로 이어질 수 있습니다.

3. **정치적 요인**: 정치적 불안정성이나 정책 변화는 환율에 큰 영향을 미칠 수 있습니다. 예를 들어, 한국의 정치적 불안정성이 커지면 외국인 투자자들이 자금을 회수할 수 있어 원화가 약세를 보일 수 있습니다.

4. **글로벌 경제 상황**: 글로벌 경제의 변화, 특히 미국의 경제 지표나 중국의 경제 상황 등도 환율에 영향을 미칠 수 있습니다. 예를 들어, 미국의 경제가 호조를 보이면 달러가 강세를 보일 수 있습니다.

5. **인플레이션**: 한국의 인플레이션이 높아지면 원화의 구매력이 떨어져 환율이 상승할 수 있습니다. 반대로, 미국의 인플레이션이 낮아지면 달러가 강세를 보일 수 있습니다.

결론적으로, 현재의 환율은 여러 경제적, 정치적 요인에 의해 영향을 받고 있으며, 앞으로의 환율 변동은 이러한 요인들의 변화에 따라 달라질 수 있습니다. 투자